In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%%capture
!pip install ftfy translate langdetect imbalanced-learn collections-extended rake_nltk googletrans

In [ ]:
import keras 
import numpy as np
from keras.preprocessing.text import Tokenizer
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, Dropout, Embedding, LSTM, Flatten
from keras.models import Model
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
plt.style.use('ggplot')
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import sys
sys.path.append('/content/drive/My Drive/Capstone')

In [5]:
filename= "/content/drive/My Drive/Capstone/final_dataframe_forDeep_Learning_Model.csv"
df = pd.read_csv(filename)

In [103]:
from keras.preprocessing.text import Tokenizer
MAX_LENGTH = 300
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df8.data.values)
post_seq = tokenizer.texts_to_sequences(df8.data.values)
post_seq_padded = pad_sequences(post_seq, maxlen=MAX_LENGTH)

In [104]:
y = df8['target']

In [107]:
X_train, X_test, y_train, y_test = train_test_split(post_seq_padded, y, test_size=0.3, random_state=1)

In [108]:
vocab_size = len(tokenizer.word_index) + 1

In [140]:
num_class = len(np.unique(df8['target'].values))

In [143]:
inputs = Input(shape=(MAX_LENGTH, ))
embedding_layer = Embedding(vocab_size,
                            128,
                            input_length=MAX_LENGTH)(inputs)

x = LSTM(64)(embedding_layer)
x = Dense(32, activation='relu')(x)
predictions = Dense(num_class, activation='softmax')(x)
model = Model(inputs=[inputs], outputs=predictions)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['acc'])

model.summary()
filepath="weights.hdf5"
checkpointer = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
history = model.fit([X_train], batch_size=64, y=to_categorical(y_train), verbose=1, validation_split=0.3, 
          shuffle=True, epochs=10, callbacks=[checkpointer])

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 300)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 300, 128)          2237056   
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense_8 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_9 (Dense)              (None, 69)                2277      
Total params: 2,290,821
Trainable params: 2,290,821
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 21585 samples, validate on 9252 samples
Epoch 1/10
21585/21585 [==============================] - 215s 10ms/step - loss: 2.7350 - acc: 0.3630 - val_loss: 1.2430 - val_acc: 0.7195

Epoch 00001: val_acc improved from -inf to 0.71952, saving model to weights.hdf5
Epoch 2/10
21585/21585 [==============================] - 212s 10ms/step - loss: 0.7493 - acc: 0.8252 - val_loss: 0.5638 - val_acc: 0.8692

Epoch 00002: val_acc improved from 0.71952 to 0.86922, saving model to weights.hdf5
Epoch 3/10
21585/21585 [==============================] - 211s 10ms/step - loss: 0.3626 - acc: 0.9133 - val_loss: 0.4875 - val_acc: 0.8931

Epoch 00003: val_acc improved from 0.86922 to 0.89310, saving model to weights.hdf5
Epoch 4/10
21585/21585 [==============================] - 211s 10ms/step - loss: 0.2258 - acc: 0.9451 - val_loss: 0.3901 - val_acc: 0.9126

Epoch 00004: val_acc improved from 0.89310 to 0.91256, saving model to weights.hdf5
Epoch 5/10
21585/21585 [==============================] - 

In [144]:

predicted = model.predict(X_test)
predicted = np.argmax(predicted, axis=1)
accuracy_score(y_test, predicted)

0.9287984261501211